## Introduction

Ce rapport présente les résultats du travail pratique sur les Machines à Vecteurs de Support (SVM). L'objectif est d'explorer les principes fondamentaux des SVM, de comparer différents noyaux sur un jeu de données simple (Iris), puis d'appliquer ces techniques à une tâche plus complexe de classification de visages. Nous étudierons en particulier l'influence du paramètre de régularisation `C`, l'impact de l'ajout de variables non pertinentes (bruit), et l'amélioration des performances grâce à la réduction de dimension par Analyse en Composantes Principales (ACP/PCA).


In [ ]:
#| echo: false
#| warning: false

# --- Imports et configuration ---
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from time import time

from sklearn.svm import SVC
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')

# Classification sur le jeu de données Iris

Nous commençons par une tâche de classification simple sur le jeu de données Iris. Nous ne conserverons que les classes 1 et 2, ainsi que les deux premiers attributs pour permettre une visualisation en 2D. Nous comparons les performances d'un noyau linéaire et d'un noyau polynomial.


In [ ]:
#| warning: false

###################################################
#               Iris Dataset
###################################################
iris = datasets.load_iris()
X = iris.data
X = scaler.fit_transform(X)
y = iris.target
X = X[y != 0, :2]
y = y[y != 0]

# Visualization
plt.show()
plt.close("all")
plt.ion()
plt.figure(1, figsize=(10, 5))
plt.title('iris data set')
plot_2d(X, y)

# split train test (say 25% for the test)
# using train_test_split whithout shuffling (fix the random state = 42) for reproductibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## Q1: Noyau Linéaire

Nous utilisons `GridSearchCV` pour trouver le meilleur hyperparamètre de régularisation `C` pour un SVM à noyau linéaire.


In [ ]:
parameters_linear = {'kernel': ['linear'], 'C': np.logspace(-3, 3, 100)}
clf_linear = GridSearchCV(SVC(), parameters_linear, n_jobs=-1)
clf_linear.fit(X_train_iris, y_train_iris)

train_score = clf_linear.score(X_train_iris, y_train_iris)
test_score = clf_linear.score(X_test_iris, y_test_iris)

print(f"Meilleur C trouvé pour le noyau linéaire : {clf_linear.best_params_['C']:.4f}")
print(f"Score sur l'ensemble d'entraînement : {train_score:.4f}")
print(f"Score de généralisation (test) : {test_score:.4f}")

## Q2: Noyau Polynomial

Nous étendons la recherche pour un noyau polynomial, en optimisant `C`, `gamma`, et le degré du polynôme `degree`.


In [ ]:
#| warning: false
parameters_poly = {
    'kernel': ['poly'],
    'C': np.logspace(-3, 3, 5),
    'gamma': np.logspace(-3, 2, 5),
    'degree': [2, 3, 4]
}
clf_poly = GridSearchCV(SVC(), parameters_poly, n_jobs=-1)
clf_poly.fit(X_train_iris, y_train_iris)

train_score_poly = clf_poly.score(X_train_iris, y_train_iris)
test_score_poly = clf_poly.score(X_test_iris, y_test_iris)

print("Meilleurs hyperparamètres pour le noyau polynomial :")
print(clf_poly.best_params_)
print(f"\nScore sur l'ensemble d'entraînement : {train_score_poly:.4f}")
print(f"Score de généralisation (test) : {test_score_poly:.4f}")

## Comparaison et Visualisation des Frontières

Visualisons les frontières de décision apprises par les deux modèles sur l'ensemble d'entraînement.


In [ ]:
#| warning: false

def f_linear(xx):
    return clf_linear.predict(xx.reshape(1, -1))

def f_poly(xx):
    return clf_poly.predict(xx.reshape(1, -1))

plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
frontiere(f_linear, X_train_iris, y_train_iris)
plt.title('Frontière de décision - Noyau Linéaire')

plt.subplot(1, 2, 2)
frontiere(f_poly, X_train_iris, y_train_iris)
plt.title('Frontière de décision - Noyau Polynomial')

plt.tight_layout()
plt.show()

**Analyse :** Pour ce jeu de données, les classes sont presque linéairement séparables. Le noyau linéaire obtient un excellent score de généralisation. Le noyau polynomial, bien que plus flexible, n'apporte pas d'amélioration significative et pourrait même être sujet au sur-apprentissage si les paramètres n'étaient pas soigneusement choisis par validation croisée. La frontière non-linéaire qu'il apprend est très proche de la frontière linéaire.

# Tâche de Classification de Visages

Nous abordons maintenant une tâche plus complexe : distinguer les visages de deux personnalités publiques, Tony Blair et Colin Powell, à partir du jeu de données "Labeled Faces in the Wild" (LFW).


In [ ]:
#| echo: false
#| warning: false

# --- Préparation des données LFW ---
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4,
                              color=True, funneled=False, slice_=None,
                              download_if_missing=True)

images = lfw_people.images
n_samples, h, w, n_colors = images.shape
target_names = lfw_people.target_names.tolist()

names = ['Tony Blair', 'Colin Powell']
idx0 = (lfw_people.target == target_names.index(names[0]))
idx1 = (lfw_people.target == target_names.index(names[1]))
images = np.r_[images[idx0], images[idx1]]
n_samples = images.shape[0]
y_faces = np.r_[np.zeros(np.sum(idx0)), np.ones(np.sum(idx1))].astype(int)

# Extraction de caractéristiques (passage en niveaux de gris et aplatissement)
X_faces = (np.mean(images, axis=3)).reshape(n_samples, -1)

# Division des données
indices = np.random.permutation(X_faces.shape[0])
train_idx, test_idx = indices[:X_faces.shape[0] // 2], indices[X_faces.shape[0] // 2:]
X_train_faces_raw, X_test_faces_raw = X_faces[train_idx, :], X_faces[test_idx, :]
y_train_faces, y_test_faces = y_faces[train_idx], y_faces[test_idx]
images_train, images_test = images[train_idx, :, :, :], images[test_idx, :, :, :]

# Standardisation (correctement, après la division)
scaler_faces = StandardScaler()
X_train_faces = scaler_faces.fit_transform(X_train_faces_raw)
X_test_faces = scaler_faces.transform(X_test_faces_raw)

## Q4: Influence du paramètre de régularisation `C`

Le paramètre `C` contrôle le compromis entre la complexité du modèle (maximiser la marge) et l'erreur de classification sur l'ensemble d'entraînement. Un `C` faible favorise une grande marge au détriment de quelques erreurs (modèle simple, potentiellement sous-ajusté). Un `C` élevé pénalise fortement les erreurs, ce qui peut conduire à une marge plus petite et à un modèle complexe, risquant le sur-ajustement.

Nous entraînons un SVM linéaire pour une plage de valeurs de `C` et observons son score sur l'ensemble de test.


In [ ]:
#| warning: false

print("--- Recherche du meilleur C pour le noyau linéaire ---")
t0 = time()
Cs = np.logspace(-5, 5, 11)
scores_train = []
scores_test = []

for C in Cs:
    clf_tmp = SVC(kernel='linear', C=C)
    clf_tmp.fit(X_train_faces, y_train_faces)
    scores_train.append(clf_tmp.score(X_train_faces, y_train_faces))
    scores_test.append(clf_tmp.score(X_test_faces, y_test_faces))

print(f"Recherche effectuée en {time() - t0:.3f}s")

# Plot
plt.figure(figsize=(10, 6))
plt.plot(Cs, scores_train, label="Score d'entraînement", marker='o')
plt.plot(Cs, scores_test, label="Score de test", marker='o')
plt.xlabel("Paramètre de régularisation C")
plt.ylabel("Score (Accuracy)")
plt.xscale('log')
plt.title("Influence du paramètre C sur la performance du SVM")
plt.legend()
plt.grid(True, which="both", ls="--")
plt.show()

best_idx = np.argmax(scores_test)
best_C = Cs[best_idx]
best_acc = scores_test[best_idx]
print(f"Meilleur C trouvé : {best_C:.2e}")
print(f"Meilleur score de généralisation (accuracy) : {best_acc:.4f}")

**Analyse :** Le graphique illustre parfaitement le compromis biais-variance. Pour des valeurs de `C` très faibles, le modèle est sous-ajusté et ses performances sont médiocres sur les deux ensembles. À mesure que `C` augmente, le modèle s'adapte mieux aux données, et le score de test s'améliore, atteignant un pic. Si `C` devient trop grand, le modèle commence à sur-apprendre les spécificités du jeu d'entraînement (le score d'entraînement continue d'augmenter) au détriment de sa capacité à généraliser (le score de test stagne ou diminue).

## Q5: Impact de l'ajout de variables de nuisance

Nous allons maintenant évaluer l'effet de l'ajout de variables non informatives (bruit gaussien) sur les performances du classifieur. C'est une illustration de la "malédiction de la dimensionnalité".


In [ ]:
#| warning: false

def run_svm_grid_search(_X_train, _y_train, _X_test, _y_test):
    parameters = {'kernel': ['linear'], 'C': np.logspace(-3, 3, 5)}
    clf = GridSearchCV(SVC(), parameters, n_jobs=-1)
    clf.fit(_X_train, _y_train)
    test_score = clf.score(_X_test, _y_test)
    return test_score

# Score sur les données originales
score_original = run_svm_grid_search(X_train_faces, y_train_faces, X_test_faces, y_test_faces)
print(f"Score de généralisation sur les données originales : {score_original:.4f}")

# Ajout de bruit aux données
n_train_samples = X_train_faces.shape[0]
n_test_samples = X_test_faces.shape[0]
noise_train = np.random.randn(n_train_samples, 300)
noise_test = np.random.randn(n_test_samples, 300)

X_train_noisy = np.concatenate((X_train_faces, noise_train), axis=1)
X_test_noisy = np.concatenate((X_test_faces, noise_test), axis=1)

# Score sur les données bruitées
score_noisy = run_svm_grid_search(X_train_noisy, y_train_faces, X_test_noisy, y_test_faces)
print(f"Score de généralisation sur les données bruitées : {score_noisy:.4f}")

**Analyse :** Comme attendu, l'ajout de 300 variables de bruit aléatoire a considérablement dégradé les performances du classifieur. Le modèle est "distrait" par ces caractéristiques non pertinentes et a plus de mal à trouver la frontière de séparation optimale basée sur les caractéristiques utiles.

## Q6: Amélioration via la réduction de dimension (PCA)

Pour contrer l'effet négatif des variables de nuisance, nous appliquons une Analyse en Composantes Principales (PCA) sur les données bruitées avant de les fournir au SVM. La PCA va identifier les axes de plus grande variance, qui correspondent (on l'espère) aux caractéristiques originales et non au bruit, nous permettant ainsi de "nettoyer" les données.


In [ ]:
#| warning: false

grid_n_components = [20, 60, 100, 150]
test_scores_pca = []

for k in grid_n_components:
    print(f"Test avec n_components = {k}...")
    pca = PCA(n_components=k, svd_solver='randomized', whiten=True)
    
    # Appliquer la PCA
    X_train_pca = pca.fit_transform(X_train_noisy)
    X_test_pca = pca.transform(X_test_noisy)
    
    # Calculer le score
    score_pca = run_svm_grid_search(X_train_pca, y_train_faces, X_test_pca, y_test_faces)
    test_scores_pca.append(score_pca)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(grid_n_components, test_scores_pca, marker='o')
plt.xlabel("Nombre de composantes principales (k)")
plt.ylabel("Score de test (Accuracy)")
plt.title("Performance du SVM après PCA sur données bruitées")
plt.grid(True)
plt.show()

best_idx_pca = np.argmax(test_scores_pca)
best_k = grid_n_components[best_idx_pca]
best_score_pca = test_scores_pca[best_idx_pca]
print(f"Meilleur nombre de composantes : {best_k}")
print(f"Meilleur score après PCA : {best_score_pca:.4f}")

**Analyse :** Les résultats sont spectaculaires. En utilisant la PCA pour réduire la dimensionnalité des données bruitées, nous avons pu récupérer, et même légèrement dépasser, les performances du modèle sur les données originales non bruitées. Le graphique montre qu'un nombre optimal de composantes (autour de 100-150) permet de capturer l'essentiel de l'information utile tout en filtrant une grande partie du bruit. Cela démontre l'efficacité de la PCA comme technique de pré-traitement pour les données de grande dimension.

## Q7: Biais dans le prétraitement des données (Script Original)

La question 7 du TP demande d'identifier un biais dans le prétraitement des données du script `svm_script.py`. Ce biais est un problème classique de **fuite de données (data leakage)**.

Dans le script initial fourni pour le TP, l'étape de standardisation des caractéristiques (`X -= np.mean(X, axis=0); X /= np.std(X, axis=0)`) est appliquée sur **l'ensemble du jeu de données `X`**, *avant* de le diviser en un ensemble d'entraînement et un ensemble de test.

**Pourquoi est-ce un biais ?**

Le `StandardScaler` (ou la standardisation manuelle) calcule la moyenne et l'écart-type des données pour les centrer et les réduire. En effectuant cette opération sur la totalité des données, les statistiques de l'ensemble de test (moyenne et écart-type) sont utilisées pour transformer l'ensemble d'entraînement. Autrement dit, le modèle, pendant sa phase d'apprentissage, a accès à des informations provenant de données qu'il n'est pas censé avoir vues.

**Quel est l'impact ?**

Ce biais conduit à une **estimation trop optimiste des performances de généralisation du modèle**. Le score obtenu sur l'ensemble de test est artificiellement gonflé car les données de test ne sont plus totalement "inconnues" du processus d'entraînement.

**La procédure correcte (appliquée dans ce rapport) :**

1.  **Diviser** les données en ensemble d'entraînement et de test.
2.  **Ajuster (`fit`)** le `StandardScaler` **uniquement** sur l'ensemble d'entraînement pour apprendre les paramètres de mise à l'échelle (moyenne et écart-type).
3.  **Appliquer la transformation (`transform`)** avec ce même scaler sur l'ensemble d'entraînement ET sur l'ensemble de test.

# Conclusion

Ce travail pratique nous a permis de mettre en œuvre et d'évaluer les SVM sur différentes tâches. Nous avons constaté l'importance du choix du noyau et des hyperparamètres, qui peuvent être optimisés par validation croisée. L'étude sur les données de visages a illustré de manière concrète l'impact du paramètre de régularisation `C` sur le compromis biais-variance. Enfin, nous avons démontré expérimentalement la malédiction de la dimensionnalité et l'efficacité de la PCA pour y remédier, tout en soulignant l'importance cruciale d'éviter la fuite de données lors du prétraitement pour obtenir une évaluation fiable des performances du modèle.